# Ch7.Callbacks - 다양한 이벤트 발생 시 처리하기
- 1. Callbacks 모듈로 할 수 있는 일 알아보기
- 2. Callbacks 모듈을 사용하여 외부 라이브러리와 연동하기
- 3. 로그를 터미널에 표시할 수 있는 Callbacks 만들기

## 1. Callbacks 모듈로 할 수 있는 일 알아보기
> 로그 수집 및 모니터링, 다른 애플리케이션과 연동 가능

<img src="./img/lang_7_1.png" width="70%" height="70%">


## 2. Callbacks 모듈을 사용하여 외부 라이브러리와 연동하기
>  chainlit에서 제공하는 랭체인과 연동을 위한 기능을 통해 callbacks모듈의 작동을 알아본다.
```
chainlit.exe run chainlit_callback.py --port 8001
```

In [3]:
'''
chainlit_callback.py 파일에서 검색agent를 어치면 에러가 발생합니다.
'''
import chainlit as cl
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatOpenAI(
    temperature=0,  
    model="gpt-4o"
)

tools = load_tools( 
    [
        "serpapi",
    ],
    serpapi_api_key=os.getenv("SERP_API_KEY")
)

agent = initialize_agent(tools=tools, llm=chat, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="Agent 초기화 완료").send() 

@cl.on_message
async def on_message(input_message):
    result = agent.run( #← Agent를 실행
        input_message, #← 입력 메시지
        callbacks=[ #← 콜백을 지정
            cl.LangchainCallbackHandler() #← chainlit에 준비된 Callbacks를 지정
        ]
    )
    await cl.Message(content=result).send()

In [ ]:
'''
callback함수를 직접 만들어서 실행도 가능 
'''

import chainlit as cl
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import warnings
import os
warnings.filterwarnings('ignore')
load_dotenv()

chat = ChatOpenAI(
    temperature=0,  
    model="gpt-3.5-turbo"
)

tools = load_tools( 
    [
        "serpapi",
    ],
    serpapi_api_key=os.getenv("SERP_API_KEY")
)

class MyCustomHandlerOne(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"on_llm_start {serialized['name']}")

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        print(f"on_new_token {token}")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:"""Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        print(f"on_chain_start {serialized['name']}")

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        print(f"on_tool_start {serialized['name']}")

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        print(f"on_agent_action {action}")

handler = MyCustomHandlerOne()
agent = initialize_agent(
    tools=tools,
    llm =chat,
    agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="Agent 초기화 완료").send() 

@cl.on_message
async def on_message(input_message):
    result = agent.run( #← Agent를 실행
        input_message, #← 입력 메시지
        callbacks=[ #← 콜백을 지정
            handler #← chainlit에 준비된 Callbacks를 지정
        ]
    )
    await cl.Message(content=result).send()

## 3. 로그를 터미널에 표시할 수 있는 Callbacks 만들기
> 이벤트 발생시 로그를 터미널에 표시할 수 있는 기능을 만들어 Callbacks모듈의 사용법을 알아보자

In [4]:
from langchain.callbacks.base import BaseCallbackHandler #← BaseCallbackHandler 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
import os
load_dotenv()

class LogCallbackHandler(BaseCallbackHandler): #← Callback을 생성

    def on_chat_model_start(self, serialized, messages, **kwargs): #← Chat models 실행 시작 시 호출되는 처리를 정의
        print("Chat models 실행 시작....")
        print(f"입력: {messages}")

    def on_chain_start(self, serialized, inputs, **kwargs): #← Chain 실행 시작 시 호출되는 처리를 정의
        print("Chain 실행 시작....")
        print(f"입력: {inputs}")

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    api_key=os.getenv('OPENAI_API_KEY'),
    callbacks=[ #← Chat models 초기화 시 Callback을 지정
        LogCallbackHandler() #← 생성한 LogCallbackHandler를 지정
    ]
)

result = chat([
    HumanMessage(content="안녕하세요!"),
])

print(result.content)

E:\miniconda\envs\lang310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Chat models 실행 시작....
입력: [[HumanMessage(content='안녕하세요!')]]
안녕하세요! 무엇을 도와드릴까요? 😊
